In [ ]:
# We are working on a templating system using python.
# We have a folder next to this notebook called `patterns`.
# The `patterns` folder children are each considered a "pattern", with the name of the folder being the name of the pattern.
# each pattern consists of its child files and directories.
# These children are modeled after the root of the project.
# The goal of the pattern system is to ask the user for a streamlined list of values, from which significant additions of the codebase will be generated.
# This is for automatically defining and registering new Bevy plugins for a rust game engine.
# Here is the current structure:
# ```
# Cursor-Hero on  main [$!?] is 📦 v0.3.0 via 🦀 v1.76.0-nightly took 2s 
# ❯ ls .\templates\patterns\ --tree
# .\templates\patterns
# ├── new_plugin_with_types
# │  ├── Cargo.toml
# │  └── crates
# │     ├── plugins
# │     │  ├── Cargo.toml
# │     │  └── src
# │     │     └── lib.rs
# │     ├── {{crate_name}}
# │     │  ├── Cargo.toml
# │     │  └── src
# │     │     ├── lib.rs
# │     │     ├── {{crate_name}}_types.rs
# │     │     └── {{crate_name}}_types_plugin.rs
# │     └── {{crate_name}}_types
# │        ├── Cargo.toml
# │        └── src
# │           ├── lib.rs
# │           └── {{crate_name}}_types_plugin.rs
# └── new_tool
#    ├── Cargo.toml
#    └── crates
#       ├── plugins
#       │  ├── Cargo.toml
#       │  └── src
#       │     └── lib.rs
#       └── {{crate_name}}_tool
#          ├── Cargo.toml
#          └── src
#             ├── lib.rs
#             └── {{crate_name}}_tool_plugin.rs
# ```
# here is the content of templates/patterns/new_plugin_with_types/Cargo.toml
# ```
# {{before_first_workspace_dependency}}
# cursor_hero_{{crate_name}}_types = { path = "./crates/{{crate_name}}_types" }
# cursor_hero_{{crate_name}} = { path = "./crates/{{crate_name}}" }
# {{first_workspace_dependency_onwards}}
# ```
# Note that this is a template for modifying the existing Cargo.toml to feature some additions.

# As you can see, The content of these files uses variables (i.e., surrounded by double-curlies) to present new files or modifications to existing files.
# For some files, like the project-wide Cargo.toml, we define a "before" and "after" section for each location we want to insert code.

# We likely want to use jinja2 templates for this.
# We have completed the first step of actually scaffolding some example templates.
# Now, we need to write some python to programmatically generate .py files with a stub method for splitting the text and returning a variable for each variable mentioned in the template.
# Currently, the only "global" variables are `crate_name` and `crate_name_pascal`, which all templates may use.

# use pathlib
# use double quotes instead of single

# global_variables = ["crate_name", "crate_name_pascal"]
# for each file in template glob **/*.*:
#     identify variable names in file by matching on {{.*}}
#     discard variable names present in global_variables
#     destination_folder = path("matchers") / file.relative_to("patterns")
#     mkdir destination_folder if not exists
#     with open(destination_folder / file.name, "w") as f:
#         inner_type = ','.join(['string']*len(variables))
#         f.write(the content of the template but commented out by adding # to the beginning of each line)
#         f.write(f"def chunk(text: str) -> Tuple[{inner_type}]:\n")
#         // now we need to scaffold the body
#         // we want to give the user a place to enter the content that they expect to preceed each variable
#         // example of what we want:
#         before_first_workspace_dependency = text before and including `[workspace.dependencies]`
#         first_workspace_dependency_onwards = text after but not including `[workspace.dependencies]`
#         return (before_first_workspace_dependency, first_workspace_dependency_onwards)
#         // but we must modify this for each template to accomodate however many variables it may have
#         // consider patterns/new_plugin_with_types
# {{use_statements}}
# use cursor_hero_{{crate_name}}::prelude::*;
# use cursor_hero_{{crate_name}}_types::prelude::*;
# {{plugin_start}}
#         app.add_plugins({{crate_name_pascal}}TypesPlugin);
#         app.add_plugins({{crate_name_pascal}}Plugin);
# {{plugin_remaining}}
#         // there are the following variables present, excluding the global ones
#         // - use_statements
#         // - plugin_start
#         // - plugin_remaining
#         // so the body for the chunk method should be:
#         def chunk(text: str) -> Tuple[string,string,string]
#             # pick one and delete this comment
#             use_statements = text before and including `replace me`
#             use_statements = text before and excluding `replace me`
#             # pick one and delete this comment
#             plugin_start = remaining text after and including `replace me`
#             plugin_start = remaining text after and excluding `replace me`
#             # pick one and delete this comment
#             plugin_remaining = remaining text after and including `replace me`
#             plugin_remaining = remaining text after and excluding `replace me`

#             return (use_statements, plugin_start, plugin_remaining)
#         // reminder: the purpose of this is so that a human can scaffold the template,
#         //   then from that scaffolding generate the python methods that will be used to
#         //   split the text into mentioned variables
#         // this will accelerate the process of creating new templates
#         // thus, we want to predict as much of what the user will need
#         // and we can include duplicate statements, indicating to the programmer that they should only keep one
#         // this involves writing the template code in such a way that adding new variables works
#         // e.g, the "remaining text" should work if they keep the inclusive statement or the exclusive statement
        
#         // reminder: this is all being generated for each file in the template
#         // the result will be a directory structure with a bunch of {{original_name_including_extension}}.py files
#         //  which can later be programmatically used to perform the template application by asking the user for the variables

# Please help me implement the code to scaffold the py files in the matchers dir, such that it mimics each template present in the patterns folder next to this jupyter notebook.
# This will be a multi-step program, so think step by step.
# Additionally, it will be a jupyter notebook.
# use two percent signs on a new line, like this, to separate cells in the notebook in your code blocks.
# %%
# you should break up the separate steps of the problem into simple variables that can be displayed for debugging purposes.
# Please include print statements and use pathlib and tqdm to make the process intuitive.
# Simple progress logging will be good for now.
# You are recommended to split new cells and print at the tail of cells often.

    

